In [5]:
from queue import PriorityQueue

def a_star(graph, start, goal, h):
    # 우선순위 큐를 초기화합니다.
    open_set = PriorityQueue()
    open_set.put((0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: h(start, goal)}
    
    while not open_set.empty():
        # f_score가 가장 낮은 노드를 꺼냅니다.
        _, current = open_set.get()
        
        # 목표에 도달했는지 확인합니다.
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]
        
        # 현재 노드의 이웃을 탐색합니다.
        for neighbor, cost in graph[current].items():
            tentative_g_score = g_score[current] + cost
            if tentative_g_score < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + h(neighbor, goal)
                open_set.put((f_score[neighbor], neighbor))
    
    return []

# 맨해튼 거리 휴리스틱 함수 (좌표 노드용)
def heuristic(node, goal):
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

# 예제 그래프 (좌표 노드 사용)
graph = {
    (0, 0): {(0, 1): 1, (1, 0): 4},
    (0, 1): {(0, 0): 1, (1, 1): 2, (0, 2): 5},
    (1, 0): {(0, 0): 4, (1, 1): 2, (2, 0): 1},
    (1, 1): {(0, 1): 2, (1, 0): 2, (1, 2): 1},
    (0, 2): {(0, 1): 5, (1, 2): 1},
    (2, 0): {(1, 0): 1, (2, 1): 2},
    (1, 2): {(1, 1): 1, (2, 2): 1},
    (2, 1): {(2, 0): 2, (2, 2): 2},
    (2, 2): {(1, 2): 1, (2, 1): 2}
}

# 시작 노드와 목표 노드
start = (0, 0)
goal = (2, 2)

# A* 알고리즘 실행
path = a_star(graph, start, goal, heuristic)
print(f"Path found: {path}")


Path found: [(0, 0), (0, 1), (1, 1), (1, 2), (2, 2)]


In [6]:
import heapq

def dijkstra(graph, start):
    # 최단 경로와 우선순위 큐를 초기화합니다.
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    priority_queue = [(0, start)]
    
    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        
        # 현재 노드까지의 거리가 이미 최단 거리보다 크면 무시합니다.
        if current_distance > distances[current_node]:
            continue
        
        # 이웃 노드를 탐색합니다.
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            # 새로운 최단 경로를 발견하면 갱신하고 큐에 추가합니다.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances

# 예제 그래프
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

# 시작 노드
start = 'A'

# 다익스트라 알고리즘 실행
distances = dijkstra(graph, start)
print(f"Shortest distances: {distances}")


Shortest distances: {'A': 0, 'B': 1, 'C': 3, 'D': 4}


In [8]:
def bellman_ford(graph, start):
    # 최단 거리를 무한대로 초기화합니다.
    distance = {node: float('inf') for node in graph}
    distance[start] = 0
    predecessor = {node: None for node in graph}
    
    # 그래프의 모든 간선을 반복합니다.
    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor, weight in graph[node].items():
                if distance[node] + weight < distance[neighbor]:
                    distance[neighbor] = distance[node] + weight
                    predecessor[neighbor] = node
    
    # 음의 사이클이 있는지 확인합니다.
    for node in graph:
        for neighbor, weight in graph[node].items():
            if distance[node] + weight < distance[neighbor]:
                # 음의 사이클 발견 시 구체적인 경로를 출력합니다.
                cycle = []
                current = node
                while current not in cycle:
                    cycle.append(current)
                    current = predecessor[current]
                cycle.append(current)
                cycle_start = cycle.index(current)
                cycle = cycle[cycle_start:]
                cycle.reverse()
                return f"Graph contains a negative weight cycle: {' -> '.join(cycle)}"
    
    return distance

# 예제 그래프 (음의 가중치 사이클 포함)
graph_with_negative_cycle = {
    'A': {'B': 1, 'C': 4},
    'B': {'C': 2, 'D': 2},
    'C': {'D': 3},
    'D': {'A': -10}
}

# 예제 그래프 (음의 가중치 사이클 미포함)
graph_without_negative_cycle = {
    'A': {'B': 1, 'C': 4},
    'B': {'C': 2, 'D': 2},
    'C': {'D': 3},
    'D': {}
}

# 시작 노드
start = 'A'

# 벨만-포드 알고리즘 실행 (음의 가중치 사이클 포함 그래프)
result = bellman_ford(graph_with_negative_cycle, start)
print(f"Result for graph with negative cycle: {result}")

# 벨만-포드 알고리즘 실행 (음의 가중치 사이클 미포함 그래프)
result = bellman_ford(graph_without_negative_cycle, start)
print(f"Result for graph without negative cycle: {result}")


Result for graph with negative cycle: Graph contains a negative weight cycle: A -> B -> D -> A
Result for graph without negative cycle: {'A': 0, 'B': 1, 'C': 3, 'D': 3}


In [9]:
def floyd_warshall(graph):
    # 초기화: 각 쌍의 최단 경로를 설정합니다.
    nodes = list(graph.keys())
    distance = {node: {node: float('inf') for node in nodes} for node in nodes}
    
    for node in nodes:
        distance[node][node] = 0
    
    for node in graph:
        for neighbor, weight in graph[node].items():
            distance[node][neighbor] = weight
    
    # 플로이드-워셜 알고리즘을 적용합니다.
    for k in nodes:
        for i in nodes:
            for j in nodes:
                if distance[i][j] > distance[i][k] + distance[k][j]:
                    distance[i][j] = distance[i][k] + distance[k][j]
    
    return distance

# 예제 그래프
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'C': 2, 'D': 5},
    'C': {'D': 1},
    'D': {}
}

# 플로이드-워셜 알고리즘 실행
distances = floyd_warshall(graph)
print("Shortest distances between every pair of nodes:")
for i in distances:
    print(f"{i}: {distances[i]}")


Shortest distances between every pair of nodes:
A: {'A': 0, 'B': 1, 'C': 3, 'D': 4}
B: {'A': inf, 'B': 0, 'C': 2, 'D': 3}
C: {'A': inf, 'B': inf, 'C': 0, 'D': 1}
D: {'A': inf, 'B': inf, 'C': inf, 'D': 0}


In [10]:
from heapq import heappop, heappush

def prim(graph, start):
    # 최소 신장 트리를 저장할 리스트와 방문한 노드를 추적할 집합을 초기화합니다.
    mst = []
    visited = set()
    edges = [(0, start)]
    
    while edges:
        weight, node = heappop(edges)
        
        # 방문하지 않은 노드만 처리합니다.
        if node not in visited:
            visited.add(node)
            mst.append((weight, node))
            
            # 이웃 노드를 탐색하여 간선을 추가합니다.
            for neighbor, cost in graph[node].items():
                if neighbor not in visited:
                    heappush(edges, (cost, neighbor))
    
    return mst

# 예제 그래프
graph = {
    'A': {'B': 1, 'C': 3, 'D': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 3, 'B': 2, 'D': 6},
    'D': {'A': 4, 'B': 5, 'C': 6},
}

# 시작 노드
start = 'A'

# 프림 알고리즘 실행
mst = prim(graph, start)
print(f"Minimum Spanning Tree: {mst}")


Minimum Spanning Tree: [(0, 'A'), (1, 'B'), (2, 'C'), (4, 'D')]
